In [1]:
import pandas as pd
import os
import dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.docstore.document import Document
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [2]:
dotenv.load_dotenv()

True

In [3]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [31]:
%pwd

'c:\\Users\\User\\Downloads\\sid_kosti\\GenAI\\iNeuron\\GenAI-Doc2QA\\research'

In [32]:
%cd ..

c:\Users\User\Downloads\sid_kosti\GenAI\iNeuron\GenAI-Doc2QA


In [33]:
%pwd

'c:\\Users\\User\\Downloads\\sid_kosti\\GenAI\\iNeuron\\GenAI-Doc2QA'

In [34]:
file_path = r"data\Siddhartha_Kosti_LLM_Resume.pdf"
loader = PyPDFLoader(file_path)
document = loader.load()

In [5]:
question_gen = ""
for doc in document:
    question_gen += doc.page_content

In [6]:
splitter_ques_gen = TokenTextSplitter(
    model_name="gpt-4o-mini",
    chunk_size=10000,
    chunk_overlap=200
)

In [7]:
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)

In [8]:
document_ques_gen = [Document(item) for item in chunk_ques_gen]

In [9]:
splitter_ans_gen = TokenTextSplitter(
    model_name = 'gpt-4o-mini',
    chunk_size = 1000,
    chunk_overlap = 100
)

In [10]:
document_answer_gen = splitter_ans_gen.split_documents(
    document_ques_gen
)

In [35]:
document_answer_gen

[Document(page_content='[Type text]  \n \n \n \n \n \nProjects  \n \n\uf0a7 LLM RAG Chatbot for Hospital system  \n\uf073 Technologies used: LangChain, Neo4j AuraDB , Agents, \nDocker, Prompt Engineering , Streamlit , Litellm . \n\uf073 Integrated LangChain and Neo4j AuraDB for data \nprocessing and storage . \n\uf073 Developed Agents and customized prompts for text -to-\nSQL functionality . \n\uf073 Dockerized the application for scalability and \ndeployment.  \n\uf073 Utilized litellm and groq -API for cost elimination  \n\uf073 Designed a user interfac e with Streamlit . \n \n\uf0a7 LLM Pipeline  evaluation  tests  \n\uf073 Hallucination and Faithfulness test  \n\uf073 Contextual relevancy test  \n\uf073 Readability & Difficulty level test  \n\uf073 Refusal and False Refusal test  \n \n\uf0a7 Golden data set & Automated  Prompt  pipeline  \n\uf073 Build  golden dataset for LLM pipeline benchmarking . \n\uf073 Implementing  automated pipeline for prompt updates . \n \n\uf0a7 RAG usin

In [11]:
llm_ques_gen_pipeline = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature = 0.3
)

c:\Users\User\anaconda3\envs\genai_qa\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [12]:
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.
Only create 5 questions.

QUESTIONS:
"""

In [13]:
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

In [14]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)

In [15]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [16]:
ques_gen_chain = load_summarize_chain(llm = llm_ques_gen_pipeline, 
                                          chain_type = "refine", 
                                          verbose = True, 
                                          question_prompt=PROMPT_QUESTIONS, 
                                          refine_prompt=REFINE_PROMPT_QUESTIONS)

In [17]:
ques = ques_gen_chain.run(document_ques_gen)

c:\Users\User\anaconda3\envs\genai_qa\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
[Type text]  
 
 
 
 
 
Projects  
 
 LLM RAG Chatbot for Hospital system  
 Technologies used: LangChain, Neo4j AuraDB , Agents, 
Docker, Prompt Engineering , Streamlit , Litellm . 
 Integrated LangChain and Neo4j AuraDB for data 
processing and storage . 
 Developed Agents and customized prompts for text -to-
SQL functionality . 
 Dockerized the application for scalability and 
deployment.  
 Utilized litellm and groq -API for cost elimination  
 Designed a user interfac e with Streamlit . 
 
 LLM Pipeline  evaluation  tests  
 Hallucination and Faithfulness test  
 Contextual relevancy test  
 Readability & Difficulty level test  
 Refus

In [19]:
embeddings = OpenAIEmbeddings()

In [20]:
vector_store = FAISS.from_documents(document_answer_gen, embeddings)

In [21]:
llm_answer_gen = ChatOpenAI(temperature=0.1, model="gpt-4o-mini")

In [22]:
ques_list = ques.split("\n")

In [23]:
ques_list

['1. **Describe the technologies used in the LLM RAG Chatbot for the hospital system. How did these technologies contribute to data processing, storage, and application deployment?**',
 '',
 '2. **What are the different evaluation tests conducted for the LLM Pipeline? Explain the significance of each test in assessing the performance of the language model.**',
 '',
 '3. **In the context of model training on a custom dataset, explain the concepts of transfer learning and fine-tuning. How did these approaches enhance performance for client applications?**',
 '',
 '4. **What methods were utilized to identify labeling consistency issues in the clustering, outlier, drift, and labeling tests? Discuss the role of K-Means and SAM (Segment Anything Model) in this process.**',
 '',
 '5. **As a Data Analyst at Raga AI Pvt Ltd, what specific tasks did you perform related to the LLM chatbot and interactive Streamlit dashboards? How did these tasks contribute to customer-specific data handling?**']

In [24]:
ques_list = [question for question in ques_list if question!=""]
ques_list

['1. **Describe the technologies used in the LLM RAG Chatbot for the hospital system. How did these technologies contribute to data processing, storage, and application deployment?**',
 '2. **What are the different evaluation tests conducted for the LLM Pipeline? Explain the significance of each test in assessing the performance of the language model.**',
 '3. **In the context of model training on a custom dataset, explain the concepts of transfer learning and fine-tuning. How did these approaches enhance performance for client applications?**',
 '4. **What methods were utilized to identify labeling consistency issues in the clustering, outlier, drift, and labeling tests? Discuss the role of K-Means and SAM (Segment Anything Model) in this process.**',
 '5. **As a Data Analyst at Raga AI Pvt Ltd, what specific tasks did you perform related to the LLM chatbot and interactive Streamlit dashboards? How did these tasks contribute to customer-specific data handling?**']

In [25]:
from langchain.chains import RetrievalQA

In [26]:
answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen, 
                                               chain_type="stuff", 
                                               retriever=vector_store.as_retriever())

In [27]:
for question in ques_list:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")

Question:  1. **Describe the technologies used in the LLM RAG Chatbot for the hospital system. How did these technologies contribute to data processing, storage, and application deployment?**
Answer:  The technologies used in the LLM RAG Chatbot for the hospital system include LangChain, Neo4j AuraDB, Docker, Prompt Engineering, Streamlit, and Litellm. 

1. **LangChain**: This technology was integrated for building applications that utilize language models. It helps in managing the flow of data and interactions between the user and the language model.

2. **Neo4j AuraDB**: This is a graph database used for data storage and processing. It allows for efficient querying and management of complex relationships in the data, which is particularly useful in a hospital system where data can be interconnected.

3. **Docker**: Docker was used to containerize the application, which enhances scalability and simplifies deployment. This means that the application can be easily deployed across differ

## final check

In [36]:
from src.helper import llm_pipeline

In [37]:
file_path = r"C:\Users\User\Downloads\sid_kosti\GenAI\iNeuron\GenAI-Doc2QA\data\Siddhartha_Kosti_LLM_Resume.pdf"

In [38]:
answer_generation_chain, ques_list = llm_pipeline(file_path)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
[Type text]  
 
 
 
 
 
Projects  
 
 LLM RAG Chatbot for Hospital system  
 Technologies used: LangChain, Neo4j AuraDB , Agents, 
Docker, Prompt Engineering , Streamlit , Litellm . 
 Integrated LangChain and Neo4j AuraDB for data 
processing and storage . 
 Developed Agents and customized prompts for text -to-
SQL functionality . 
 Dockerized the application for scalability and 
deployment.  
 Utilized litellm and groq -API for cost elimination  
 Designed a user interfac e with Streamlit . 
 
 LLM Pipeline  evaluation  tests  
 Hallucination and Faithfulness test  
 Contextual relevancy test  
 Readability & Difficulty level test  
 Refus

In [40]:
for question in ques_list:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")

Question:  1. **Technologies and Integration**: Describe how you integrated LangChain and Neo4j AuraDB in the LLM RAG Chatbot project. What role did each technology play in data processing and storage?
Answer:  In the LLM RAG Chatbot project, LangChain was utilized for building the language model capabilities and facilitating interactions with the data, while Neo4j AuraDB served as the database for data storage and processing. 

LangChain helped in creating the framework for the chatbot, enabling it to process natural language queries and generate responses based on the data stored in the database. It also facilitated the development of agents and customized prompts for text-to-SQL functionality, allowing the chatbot to convert user queries into SQL queries to retrieve relevant data.

On the other hand, Neo4j AuraDB was used for storing the data in a graph database format, which is particularly effective for handling complex relationships and connections within the data. This integrati